Machine Leanring Engineer Nanodegree Udacity
---
-------
Captonse :  Flight Delay Prediction
--------
---

This MLND Captose Project. In this we will use DOT's data set. 

### 1 - Load All The Libraries

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
import seaborn as sns
from datetime import datetime as dt  
from utils.MlUtils import MlUtils ,convert_time ,concat_date_time ,concat_date_time_ARRIVAL, concat_date_time_TaxiOut
from matplotlib.gridspec import GridSpec

### 2 -  Load all the dataset
- Pandas load csv

In [85]:
ut=MlUtils()
dfFlight = ut.load_csv('././data/flights.csv',  5000)
dfFlight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### 3 - Data Pre-Processing

In [86]:
dfFlight.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME           int64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

### `Percentage of missing values per columns`

In [87]:
ut.missingPercent(dfFlight)

                Feature  Missing_Count  Missing_%
0                  YEAR              0       0.00
1                 MONTH              0       0.00
2                   DAY              0       0.00
3           DAY_OF_WEEK              0       0.00
4               AIRLINE              0       0.00
5         FLIGHT_NUMBER              0       0.00
6           TAIL_NUMBER              6       0.12
7        ORIGIN_AIRPORT              0       0.00
8   DESTINATION_AIRPORT              0       0.00
9   SCHEDULED_DEPARTURE              0       0.00
10       DEPARTURE_TIME            285       5.70
11      DEPARTURE_DELAY            285       5.70
12             TAXI_OUT            286       5.72
13           WHEELS_OFF            286       5.72
14       SCHEDULED_TIME              0       0.00
15         ELAPSED_TIME            297       5.94
16             AIR_TIME            297       5.94
17             DISTANCE              0       0.00
18            WHEELS_ON            288       5.76


In [88]:
print(dfFlight.shape)
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove = ['FLIGHT_NUMBER', 'TAIL_NUMBER', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
                       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)


(5000, 31)
(5000, 23)


In [89]:
# dfFlight=dfFlight.iloc[:10,:]
# dfFlight.columns
dfFlight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,2015,1,1,4,AS,ANC,SEA,5,2354.0,-11.0,...,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,LAX,PBI,10,2.0,-8.0,...,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,SFO,CLT,20,18.0,-2.0,...,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,LAX,MIA,20,15.0,-5.0,...,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,SEA,ANC,25,24.0,-1.0,...,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0


#### Arriaval Delay is difference between 	`SCHEDULED_ARRIVAL`  and `ARRIVAL_TIME` 

In [90]:
t3=dt.now()
dfFlight['FLIGHT_DATE'] = pd.to_datetime(dfFlight[['YEAR', 'MONTH', 'DAY']])
print(dt.now() -t3)

0:00:00.012015


In [91]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['YEAR', 'MONTH', 'DAY']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(5000, 21)


In [92]:
t2=dt.now()
dfFlight['FLIGHT_DELAY'] = dfFlight['ARRIVAL_DELAY'].map( lambda x : 'YES' if x>0 else 'NO')
print(dfFlight.shape)
print(dt.now() -t2)

(5000, 22)
0:00:00.004012


In [93]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY
0,4,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205,...,1448,404.0,4.0,430,408.0,-22.0,0,0,2015-01-01,NO
1,4,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280,...,2330,737.0,4.0,750,741.0,-9.0,0,0,2015-01-01,NO
2,4,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286,...,2296,800.0,11.0,806,811.0,5.0,0,0,2015-01-01,YES
3,4,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285,...,2342,748.0,8.0,805,756.0,-9.0,0,0,2015-01-01,NO
4,4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235,...,1448,254.0,5.0,320,259.0,-21.0,0,0,2015-01-01,NO


In [94]:
t4=dt.now()
dfFlight['SCHEDULED_DEPARTURE'] = dfFlight.apply(concat_date_time, axis=1)
print(dt.now() -t4)

0:00:00.863796


In [95]:
t5=dt.now()
dfFlight['DEPARTURE_TIME'] = dfFlight['DEPARTURE_TIME'].map(convert_time)
print(dt.now() -t5)

0:00:00.479878


In [96]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205,...,1448,404.0,4.0,430,408.0,-22.0,0,0,2015-01-01,NO
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280,...,2330,737.0,4.0,750,741.0,-9.0,0,0,2015-01-01,NO
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286,...,2296,800.0,11.0,806,811.0,5.0,0,0,2015-01-01,YES
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285,...,2342,748.0,8.0,805,756.0,-9.0,0,0,2015-01-01,NO
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235,...,1448,254.0,5.0,320,259.0,-21.0,0,0,2015-01-01,NO


In [97]:
t6=dt.now()
dfFlight['SCHEDULED_ARRIVAL'] =  dfFlight.apply(concat_date_time_ARRIVAL, axis=1)
print(dfFlight.columns)
print(dfFlight.shape)
print(dt.now() -t6)

Index(['DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'FLIGHT_DATE',
       'FLIGHT_DELAY'],
      dtype='object')
(5000, 22)
0:00:00.847792


In [98]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205,...,1448,404.0,4.0,2015-01-01 04:30:00,408.0,-22.0,0,0,2015-01-01,NO
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280,...,2330,737.0,4.0,2015-01-01 07:50:00,741.0,-9.0,0,0,2015-01-01,NO
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286,...,2296,800.0,11.0,2015-01-01 08:06:00,811.0,5.0,0,0,2015-01-01,YES
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285,...,2342,748.0,8.0,2015-01-01 08:05:00,756.0,-9.0,0,0,2015-01-01,NO
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235,...,1448,254.0,5.0,2015-01-01 03:20:00,259.0,-21.0,0,0,2015-01-01,NO


In [99]:
# Total planned Journery Time 
dfFlight['TOTAL_JOURNERY_TIME']=dfFlight['SCHEDULED_ARRIVAL']- dfFlight['SCHEDULED_DEPARTURE']

In [100]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,FLIGHT_DATE,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,15.0,205,...,404.0,4.0,2015-01-01 04:30:00,408.0,-22.0,0,0,2015-01-01,NO,04:25:00
1,4,AA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,14.0,280,...,737.0,4.0,2015-01-01 07:50:00,741.0,-9.0,0,0,2015-01-01,NO,07:40:00
2,4,US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,34.0,286,...,800.0,11.0,2015-01-01 08:06:00,811.0,5.0,0,0,2015-01-01,YES,07:46:00
3,4,AA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,30.0,285,...,748.0,8.0,2015-01-01 08:05:00,756.0,-9.0,0,0,2015-01-01,NO,07:45:00
4,4,AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,35.0,235,...,254.0,5.0,2015-01-01 03:20:00,259.0,-21.0,0,0,2015-01-01,NO,02:55:00


### Percentage of missing values per columns

In [101]:
ut.missingPercent(dfFlight)

                Feature  Missing_Count  Missing_%
0           DAY_OF_WEEK              0       0.00
1               AIRLINE              0       0.00
2        ORIGIN_AIRPORT              0       0.00
3   DESTINATION_AIRPORT              0       0.00
4   SCHEDULED_DEPARTURE              0       0.00
5        DEPARTURE_TIME            285       5.70
6       DEPARTURE_DELAY            285       5.70
7              TAXI_OUT            286       5.72
8            WHEELS_OFF            286       5.72
9        SCHEDULED_TIME              0       0.00
10         ELAPSED_TIME            297       5.94
11             AIR_TIME            297       5.94
12             DISTANCE              0       0.00
13            WHEELS_ON            288       5.76
14              TAXI_IN            288       5.76
15    SCHEDULED_ARRIVAL              0       0.00
16         ARRIVAL_TIME            288       5.76
17        ARRIVAL_DELAY            297       5.94
18             DIVERTED              0       0.00


### Removing Features which are not more relevant to our model 
___
- `ARRIVAL_DELAY` : Drived `FLIGHT_DELAY`
- `YEAR`, `MONTH`, `DAY` : We drived anohter feature by using them, So i am going to remove them.
- `FLIGHT_NUMBER` : It's flight number like other vehical number. it
- `TAIL_NUMBER` : It's unique identifier for Aircraft. It's called Aircart registration number. 
- `CANCELLATION_REASON` , `AIR_SYSTEM_DELAY` , `SECURITY_DELAY`, `AIRLINE_DELAY`, `LATE_AIRCRAFT_DELAY`, `WEATHER_DELAY` :  Aprox 82% value are missing for these features. 
- `AIR_TIME` : This will be populated after landing. So it's not important feature for our model.
- `FLIGHT_DATE` : Not relevent 
- `ARRIVAL_TIME` ,`WHEELS_ON`, `TAXI_IN` , `CANCELLED` : This will be populated after landing on airport.

- `WHEELS_ON` : The time point that the aircraft's wheels touch on the ground
- `TAXI_IN` : The time duration elapsed between wheels-on and gate arrival at the destination airport
- `ARRIVAL_TIME` : WHEELS_ON+TAXI_IN

- `ELAPSED_TIME` : AIR_TIME+TAXI_IN+TAXI_OUT
 

In [102]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['ARRIVAL_DELAY','ELAPSED_TIME' , 'AIR_TIME', 'SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL',
                     'FLIGHT_DATE','ARRIVAL_TIME' , 'WHEELS_ON', 'TAXI_IN','CANCELLED'] 
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(5000, 13)


In [103]:
dfFlight.columns

Index(['DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'DISTANCE', 'DIVERTED', 'FLIGHT_DELAY',
       'TOTAL_JOURNERY_TIME'],
      dtype='object')

In [104]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,23:54:00,-11.0,21.0,15.0,205,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,00:02:00,-8.0,12.0,14.0,280,2330,0,NO,07:40:00
2,4,US,SFO,CLT,00:18:00,-2.0,16.0,34.0,286,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,00:15:00,-5.0,15.0,30.0,285,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,00:24:00,-1.0,11.0,35.0,235,1448,0,NO,02:55:00


In [105]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['DEPARTURE_TIME','SCHEDULED_TIME','WHEELS_OFF'] 
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(5000, 10)


In [106]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,-11.0,21.0,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,-8.0,12.0,2330,0,NO,07:40:00
2,4,US,SFO,CLT,-2.0,16.0,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,-5.0,15.0,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,-1.0,11.0,1448,0,NO,02:55:00


### Data verification realated to Imbalance or Balance
----

In [107]:
# Calculate Count of classes per class
dfDelayClass=dfFlight['FLIGHT_DELAY'].T.value_counts().reset_index() 
dfDelayClass.columns=["FLIGHT_DELAY_CLASS","COUNT_PER_CLASS"]
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS
0,NO,3423
1,YES,1577


In [108]:
dfDelayClass['%_PER_CLASS']=dfDelayClass['COUNT_PER_CLASS'].map(ut.Missing_Count)
# dMissing_CountfDelayClass
dfDelayClass

,FLIGHT_DELAY_CLASS,COUNT_PER_CLASS,%_PER_CLASS
0,NO,3423,68.46
1,YES,1577,31.54


In [109]:
ut.missingPercent(dfFlight=dfFlight)

               Feature  Missing_Count  Missing_%
0          DAY_OF_WEEK              0       0.00
1              AIRLINE              0       0.00
2       ORIGIN_AIRPORT              0       0.00
3  DESTINATION_AIRPORT              0       0.00
4      DEPARTURE_DELAY            285       5.70
5             TAXI_OUT            286       5.72
6             DISTANCE              0       0.00
7             DIVERTED              0       0.00
8         FLIGHT_DELAY              0       0.00
9  TOTAL_JOURNERY_TIME              0       0.00
0:00:00.011996


In [110]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,4,AS,ANC,SEA,-11.0,21.0,1448,0,NO,04:25:00
1,4,AA,LAX,PBI,-8.0,12.0,2330,0,NO,07:40:00
2,4,US,SFO,CLT,-2.0,16.0,2296,0,YES,07:46:00
3,4,AA,LAX,MIA,-5.0,15.0,2342,0,NO,07:45:00
4,4,AS,SEA,ANC,-1.0,11.0,1448,0,NO,02:55:00


### Airline Details 
---


In [111]:
# Converting Day_of_week in Catogical Feature
week_Days={1:'ONE',2:'TWO',3:'THREE',4:'FOUR',5:'FIVE',6:'SIX',7:'SEVEN'}
delay_binary={'YES':0,'NO':1}

In [112]:
dfFlight['DAY_OF_WEEK']=dfFlight['DAY_OF_WEEK'].map(week_Days)
dfFlight['FLIGHT_DELAY']=dfFlight['FLIGHT_DELAY'].map(delay_binary)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME
0,FOUR,AS,ANC,SEA,-11.0,21.0,1448,0,1,04:25:00
1,FOUR,AA,LAX,PBI,-8.0,12.0,2330,0,1,07:40:00
2,FOUR,US,SFO,CLT,-2.0,16.0,2296,0,0,07:46:00
3,FOUR,AA,LAX,MIA,-5.0,15.0,2342,0,1,07:45:00
4,FOUR,AS,SEA,ANC,-1.0,11.0,1448,0,1,02:55:00


#### Feature Engineering 
---
`We will convert all the time in minutes in Decimal`

- `ORIGIN_AIRPORT` `DESTINATION_AIRPORT` : `LAG_INFO`


In [113]:
dfFlight['LAG_INFO']= dfFlight['ORIGIN_AIRPORT']+ '--' +dfFlight['DESTINATION_AIRPORT']
dfFlight.shape

(5000, 11)

In [114]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,ANC,SEA,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,LAX,PBI,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,SFO,CLT,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,LAX,MIA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,SEA,ANC,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


In [115]:
# Removing because  We have drive the Flight_Delay  from  ARRIVAL_DELAY. So Arrival_Delay is no relevent more.
variables_to_remove=['DESTINATION_AIRPORT','ORIGIN_AIRPORT']
dfFlight.drop(variables_to_remove, axis=1, inplace=True)
print(dfFlight.shape)

(5000, 9)


In [116]:
print(dfFlight['LAG_INFO'].nunique())
dfFlight.head()

2645


,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


- departure timeing AM/PM
- `SCHEDULED_DEPARTURE` : Planned Departure Time
- `DEPARTURE_TIME` : WHEEL_OFF - TAXI_OUT
- `DEPARTURE_DELAY` : Total Delay on Departure in Minutes
- `TAXI_OUT` : The time duration elapsed between departure from the origin airport gate and wheels off
- `WHEELS_OFF` : The time point that the aircraft's wheels leave the ground
- `SCHEDULED_TIME` : Planned time amount needed for the flight trip
- `AIR_TIME` : The time duration between wheels_off and wheels_on time
- `DISTANCE` : Distance between two airports

- `SCHEDULED_ARRIVAL` : Planned arrival time

- `ARRIVAL_DELAY` : ARRIVAL_TIME-SCHEDULED_ARRIVAL
- `DIVERTED` : Aircraft landed on airport that out of schedule

In [117]:
# convert time to decimal minutes

#     rawtime = pd.to_datetime(t, format='%H:%M:%S').time() 
#     print(t.hour, rawtime.minute)
#      m = (hours * 60) + minutes + (seconds / 60). Ti
#     times_in_decimal=(t.hour * 3600) + (t.minute * 60) 
def convert_hour_to_decimal_minute(rawtime):
    if pd.isna(rawtime) :
        np.nan
    else:
#         print(type(rawtime),rawtime)
        times_in_decimal = (rawtime.hour * 60) + rawtime.minute 
        #     print(times_in_decimal)
        return times_in_decimal

In [118]:
# dfTest=dfFlight.loc[:5,:]
# dfFlight['DEPARTURE_TIME_DECIMAL']=dfFlight['DEPARTURE_TIME'].map(convert_hour_to_decimal_minute)
# dfFlight.shape

In [119]:
# dfFlight = dfFlight.reindex(sorted(dfFlight.columns), axis=1)
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


In [120]:
dfFlight.dtypes

DAY_OF_WEEK                     object
AIRLINE                         object
DEPARTURE_DELAY                float64
TAXI_OUT                       float64
DISTANCE                         int64
DIVERTED                         int64
FLIGHT_DELAY                     int64
TOTAL_JOURNERY_TIME    timedelta64[ns]
LAG_INFO                        object
dtype: object

In [121]:
# dfFlight['SCHEDULED_ARRIVAL_DECIMAL']=dfFlight['SCHEDULED_ARRIVAL'].map(convert_hour_to_decimal_minute)

In [122]:
dfFlight.head()

,DAY_OF_WEEK,AIRLINE,DEPARTURE_DELAY,TAXI_OUT,DISTANCE,DIVERTED,FLIGHT_DELAY,TOTAL_JOURNERY_TIME,LAG_INFO
0,FOUR,AS,-11.0,21.0,1448,0,1,04:25:00,ANC--SEA
1,FOUR,AA,-8.0,12.0,2330,0,1,07:40:00,LAX--PBI
2,FOUR,US,-2.0,16.0,2296,0,0,07:46:00,SFO--CLT
3,FOUR,AA,-5.0,15.0,2342,0,1,07:45:00,LAX--MIA
4,FOUR,AS,-1.0,11.0,1448,0,1,02:55:00,SEA--ANC


In [123]:
df=pd.get_dummies(dfFlight, drop_first=False)
df.columns

Index(['DEPARTURE_DELAY', 'TAXI_OUT', 'DISTANCE', 'DIVERTED', 'FLIGHT_DELAY',
       'TOTAL_JOURNERY_TIME', 'DAY_OF_WEEK_FOUR', 'AIRLINE_AA', 'AIRLINE_AS',
       'AIRLINE_B6',
       ...
       'LAG_INFO_VEL--SLC', 'LAG_INFO_VPS--ATL', 'LAG_INFO_VPS--DFW',
       'LAG_INFO_VPS--IAH', 'LAG_INFO_WRG--PSG', 'LAG_INFO_XNA--ATL',
       'LAG_INFO_XNA--DEN', 'LAG_INFO_XNA--DFW', 'LAG_INFO_XNA--ORD',
       'LAG_INFO_YUM--PHX'],
      dtype='object', length=2666)

In [124]:
df.shape

(5000, 2666)

# Convert all hour to miniitutes and all minitus to decimal
----

# Model Training and Test

In [79]:
dfAirline=ut.load_csv('././data/airlines.csv',numRow=300)

In [ ]:
airline_companies = dfAirline.set_index('IATA_CODE')['AIRLINE'].to_dict()

In [ ]:
airline_companies

In [ ]:
# dfFlight(['FLIGHT_DELAY','AIRLINE']).groupby(['AIRLINE']).agg('count')
dfGroup=dfFlight.groupby(['AIRLINE','FLIGHT_DELAY'])

In [ ]:
dfGroup=dfGroup['FLIGHT_DELAY'].agg(['count']).reset_index()

In [ ]:
dfGroup

In [ ]:
dfGroup
dfGroup.columns=['AIRLINE','FLIGHT_DELAY','DELAY_COUNT']

In [ ]:
dfGroup['DELAY_COUNT_%']=dfGroup['DELAY_COUNT'].map(missing_percent)

#### Delay Analysis in respect to Air line 

In [ ]:
dfGroup['AIRLINE']=dfGroup['AIRLINE'].map(airline_companies)

In [ ]:
dfGroup

In [ ]:
# function that extract statistical parameters from a grouby objet:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
#_______________________________________________________________
# Creation of a dataframe with statitical infos on each airline:
global_stats = dfFlight['DEPARTURE_DELAY'].groupby(dfFlight['AIRLINE']).apply(get_stats).unstack()
global_stats = global_stats.sort_values('count')
global_stats

In [ ]:
font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 15}
mpl.rc('font', **font)

In [ ]:
#__________________________________________________________________
# I extract a subset of columns and redefine the airlines labeling 
df2 = dfFlight.loc[:, ['AIRLINE', 'DEPARTURE_DELAY']]
df2['AIRLINE'] = df2['AIRLINE'].replace(airline_companies)
#________________________________________________________________________
colors = ['royalblue', 'grey', 'wheat', 'c', 'firebrick', 'seagreen', 'lightskyblue',
          'lightcoral', 'yellowgreen', 'gold', 'tomato', 'violet', 'aquamarine', 'chartreuse']
#___________________________________
fig = plt.figure(1, figsize=(16,15))
gs=GridSpec(2,2)             
ax1=fig.add_subplot(gs[0,0]) 
ax2=fig.add_subplot(gs[0,1]) 
ax3=fig.add_subplot(gs[1,:]) 
#------------------------------
# Pie chart nº1: nb of flights
#------------------------------
labels = [s for s in  global_stats.index]
sizes  = global_stats['count'].values
explode = [0.3 if sizes[i] < 20000 else 0.0 for i in range(len(airline_companies))]
patches, texts, autotexts = ax1.pie(sizes, explode = explode,
                                labels=labels, colors = colors,  autopct='%1.0f%%',
                                shadow=False, startangle=0)
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax1.axis('equal')
ax1.set_title('% of flights per company', bbox={'facecolor':'midnightblue', 'pad':5},
              color = 'w',fontsize=18)
#_______________________________________________
# I set the legend: abreviation -> airline name
comp_handler = []
for i in range(len(airline_companies)):
    comp_handler.append(mpatches.Patch(color=colors[i],
            label = global_stats.index[i] + ': ' + airline_companies[global_stats.index[i]]))
ax1.legend(handles=comp_handler, bbox_to_anchor=(0.2, 0.9), 
           fontsize = 13, bbox_transform=plt.gcf().transFigure)
#----------------------------------------
# Pie chart nº2: mean delay at departure
#----------------------------------------
sizes  = global_stats['mean'].values
sizes  = [max(s,0) for s in sizes]
explode = [0.0 if sizes[i] < 20000 else 0.01 for i in range(len(airline_companies))]
patches, texts, autotexts = ax2.pie(sizes, explode = explode, labels = labels,
                                colors = colors, shadow=False, startangle=0,
                                autopct = lambda p :  '{:.0f}'.format(p * sum(sizes) / 100))
for i in range(len(airline_companies)): 
    texts[i].set_fontsize(14)
ax2.axis('equal')
ax2.set_title('Mean delay at origin', bbox={'facecolor':'midnightblue', 'pad':5},
              color='w', fontsize=18)
#------------------------------------------------------
# striplot with all the values reported for the delays
#___________________________________________________________________
# I redefine the colors for correspondance with the pie charts
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
#___________________________________________________________________
ax3 = sns.stripplot(y="AIRLINE", x="DEPARTURE_DELAY", size = 4, palette = colors,
                    data=df2, linewidth = 0.5,  jitter=True)
plt.setp(ax3.get_xticklabels(), fontsize=14)
plt.setp(ax3.get_yticklabels(), fontsize=14)
ax3.set_xticklabels(['{:2.0f}h{:2.0f}m'.format(*[int(y) for y in divmod(x,60)])
                         for x in ax3.get_xticks()])
plt.xlabel('Departure delay', fontsize=18, bbox={'facecolor':'midnightblue', 'pad':5},
           color='w', labelpad=20)
ax3.yaxis.label.set_visible(False)
#________________________
plt.tight_layout(w_pad=3) 